# SEQUENTIAL PATTERN MINING

In [51]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import OrderedDict
from gsp import apriori

***

### UTILITY

In [2]:
# utility to transform list of dicts into list of lists of transactions, given a particular customer
def getCustomerTransactions(cust_trans_dates,customer):
    cust_dict_list = cust_trans_dates[customer]
    cust_trans_list = list()
    for _,v in cust_dict_list.items():
        cust_trans_list.append(v)
    return cust_trans_list

***

In [3]:
# load dataframe
df = pd.read_csv('datasets/cleaned_dataframe.csv', sep='\t', index_col=0)
df.head()

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,536365,2010-12-01 08:26:00,2.55,17850,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,2010-12-01 08:26:00,3.39,17850,United Kingdom,71053,WHITE METAL LANTERN,6
2,536365,2010-12-01 08:26:00,2.75,17850,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,536365,2010-12-01 08:26:00,3.39,17850,United Kingdom,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,2010-12-01 08:26:00,3.39,17850,United Kingdom,84029E,RED WOOLLY HOTTIE WHITE HEART.,6


### CUSTOMER ANALYSIS

Not all customers are good for these kind of analysis, especially the customers with little orders! If the number of orders is low then mining patterns becomes almost unfeasible

In [4]:
# load customer dataframe
dfc = pd.read_csv('datasets/customer_dataframe.csv', sep='\t', index_col=0)
dfc.head()

,TProd,DProd,MeanProdOrder,TSale,MinPSale,MaxPSale,MeanSaleOrder,TRProd,MeanPSale,TOrder,SETSaleQta,SESaleQtaOrder,MeanTimeGap,MaxOrderMonth,MaxOrderDay
CustomerID,,,,,,,,,,,,,,,
17850,1702,21,48.629,5317.89,6.36,107.25,151.940,31,3.960,34,4.137,3.186,2.088,Dec,Thu
13047,1355,105,84.688,3089.10,6.64,68.00,193.069,35,3.926,9,5.421,3.875,21.800,Aug,Wed
12583,4978,114,292.824,6629.34,6.80,132.80,389.961,50,2.140,15,5.804,4.087,23.188,Sep,Fri
13748,439,24,87.800,948.25,9.36,204.00,189.650,0,3.996,5,4.080,2.322,69.500,Apr,Mon
15100,58,1,9.667,635.10,175.20,350.40,105.850,22,10.950,3,2.252,2.252,8.600,Dec,Wed


In [5]:
print("Total amount of customers:",len(dfc['TOrder']))
print("Total amount of customers with < 5 orders:",len(dfc[dfc['TOrder'] < 5]))
print("Total amount of customers with < 4 orders:",len(dfc[dfc['TOrder'] < 4]))
print("Total amount of customers with < 3 orders:",len(dfc[dfc['TOrder'] < 3]))
# here we can decide which ones to prune, < 5 can be good maybe
to_prune = dfc[dfc['TOrder']<5].index

Total amount of customers: 4333
Total amount of customers with < 5 orders: 3232
Total amount of customers with < 4 orders: 2837
Total amount of customers with < 3 orders: 2335


In [6]:
# prune away all irrelevant customers
df = df[~df['CustomerID'].isin(to_prune)]

In [7]:
df['BasketDate'] = pd.to_datetime(df["BasketDate"], dayfirst=True)

#### It is easy to notice that orders are always made on same day (differ at max by one minute intraorder)

In [8]:
"cust_trans_dates = {customer: {date: [[item1][item2]]}}"
"cust_trans = {customer: [[item1][item2]]}"
cust_trans_dates = {}
cust_trans = {}
for customer in tqdm(df['CustomerID'].unique(), total=len(df['CustomerID'].unique()), desc="Iterating over customers"):
    cust_trans_ord_dict = OrderedDict()
    cust_trans_list = list()
    cust_df = df.loc[df['CustomerID'] == customer,['BasketID', 'BasketDate', 'ProdID']]
    for basket in cust_df['BasketID'].unique():
        prod_list = cust_df[cust_df['BasketID'] == basket]['ProdID'].unique().tolist() #REMINDER FOR MYSELF: IS IT CORRECT TO MAINTAIN IN A TRANSACTION ONLY UNIQUE PRODIDS, NO REPETITIONS? FROM WHAT I SEE THIS SEEMS TO BE THE CASE BUT TRY TO SEARCH FOR CONFIRMATION
        date = cust_df[cust_df['BasketID'] == basket]['BasketDate'].unique()[0] #because of what said above we can take first date of order (at max we will have 2 elements differing of 1 minute)
        cust_trans_ord_dict[date] = prod_list
        cust_trans_list.append(prod_list)
    cust_trans_dates[customer] = cust_trans_ord_dict
    cust_trans[customer] = cust_trans_list

In [9]:
# usage example for date including structure
getCustomerTransactions(cust_trans_dates,17850)

[['85123A', '71053', '84406B', '84029G', '84029E', '22752', '21730'],
 ['22633', '22632'],
 ['22632', '22633'],
 ['85123A',
  '71053',
  '84406B',
  '20679',
  '37370',
  '21871',
  '21071',
  '21068',
  '82483',
  '82486',
  '82482',
  '82494L',
  '84029G',
  '84029E',
  '22752',
  '21730'],
 ['85123A',
  '71053',
  '84406B',
  '20679',
  '37370',
  '21871',
  '21071',
  '21068',
  '82483',
  '82486',
  '82482',
  '82494L',
  '84029G',
  '84029E',
  '22752',
  '21730'],
 ['22632', '22633'],
 ['85123A',
  '71053',
  '84406B',
  '15056BL',
  '20679',
  '37370',
  '21871',
  '21071',
  '21068',
  '82483',
  '82486',
  '82482',
  '82494L',
  '84029G',
  '84029E',
  '22752',
  '22803',
  '21730'],
 ['22632', '22633'],
 ['85123A',
  '71053',
  '84406B',
  '20679',
  '37370',
  '21871',
  '21071',
  '21068',
  '82483',
  '82486',
  '82482',
  '82494L',
  '84029G',
  '84029E',
  '22752',
  '22803',
  '21730'],
 ['22632', '22633'],
 ['85123A',
  '71053',
  '82483',
  '82486',
  '82482',
  '824

### RESULT ANALYSIS

Using a modified version (for a more optimized exectution) of gsp.py provided by Francesca, we searched for sequential patterns with different supports:
- 90
- 120
- 150
- 200

In [10]:
seq_list_90 = [([['15036']], 91), ([['15056N']], 91), ([['16161P']], 91), ([['20676']], 91), ([['20685']], 91), ([['20711']], 91), ([['20712']], 91), ([['20713']], 91), ([['20717']], 91), ([['20718']], 91), ([['20719']], 91), ([['20723']], 91), ([['20724']], 91), ([['20725']], 91), ([['20726']], 91), ([['20727']], 91), ([['20728']], 91), ([['20749']], 91), ([['20750']], 91), ([['20754']], 91), ([['20914']], 91), ([['20971']], 91), ([['20972']], 91), ([['20973']], 91), ([['20974']], 91), ([['20975']], 91), ([['21034']], 91), ([['21080']], 91), ([['21086']], 91), ([['21094']], 91), ([['21121']], 91), ([['21122']], 91), ([['21136']], 91), ([['21154']], 91), ([['21155']], 91), ([['21156']], 91), ([['21164']], 91), ([['21165']], 91), ([['21166']], 91), ([['21169']], 91), ([['21172']], 91), ([['21174']], 91), ([['21175']], 91), ([['21181']], 91), ([['21210']], 91), ([['21212']], 91), ([['21213']], 91), ([['21216']], 91), ([['21217']], 91), ([['21231']], 91), ([['21232']], 91), ([['21314']], 91), ([['21340']], 91), ([['21430']], 91), ([['21479']], 91), ([['21481']], 91), ([['21484']], 91), ([['21485']], 91), ([['21497']], 91), ([['21498']], 91), ([['21506']], 91), ([['21523']], 91), ([['21524']], 91), ([['21533']], 91), ([['21535']], 91), ([['21558']], 91), ([['21559']], 91), ([['21621']], 91), ([['21668']], 91), ([['21669']], 91), ([['21670']], 91), ([['21671']], 91), ([['21672']], 91), ([['21673']], 91), ([['21731']], 91), ([['21733']], 91), ([['21754']], 91), ([['21755']], 91), ([['21756']], 91), ([['21770']], 91), ([['21790']], 91), ([['21791']], 91), ([['21843']], 91), ([['21868']], 91), ([['21876']], 91), ([['21877']], 91), ([['21889']], 91), ([['21891']], 91), ([['21892']], 91), ([['21907']], 91), ([['21908']], 91), ([['21914']], 91), ([['21915']], 91), ([['21916']], 91), ([['21918']], 91), ([['21928']], 91), ([['21929']], 91), ([['21930']], 91), ([['21931']], 91), ([['21932']], 91), ([['21933']], 91), ([['21936']], 91), ([['21937']], 91), ([['21975']], 91), ([['21976']], 91), ([['21977']], 91), ([['21980']], 91), ([['21982']], 91), ([['21985']], 91), ([['22027']], 91), ([['22029']], 91), ([['22030']], 91), ([['22037']], 91), ([['22077']], 91), ([['22079']], 91), ([['22082']], 91), ([['22083']], 91), ([['22084']], 91), ([['22086']], 91), ([['22090']], 91), ([['22111']], 91), ([['22112']], 91), ([['22113']], 91), ([['22114']], 91), ([['22138']], 91), ([['22139']], 91), ([['22141']], 91), ([['22142']], 91), ([['22144']], 91), ([['22147']], 91), ([['22148']], 90), ([['22149']], 91), ([['22150']], 91), ([['22151']], 91), ([['22158']], 91), ([['22171']], 91), ([['22173']], 91), ([['22178']], 91), ([['22189']], 91), ([['22191']], 91), ([['22192']], 91), ([['22193']], 91), ([['22197']], 91), ([['22212']], 91), ([['22219']], 91), ([['22271']], 91), ([['22273']], 91), ([['22294']], 91), ([['22296']], 91), ([['22297']], 91), ([['22326']], 91), ([['22328']], 91), ([['22333']], 91), ([['22352']], 91), ([['22355']], 91), ([['22356']], 91), ([['22367']], 91), ([['22379']], 91), ([['22382']], 91), ([['22383']], 91), ([['22384']], 91), ([['22385']], 91), ([['22386']], 91), ([['22398']], 90), ([['22411']], 91), ([['22413']], 91), ([['22417']], 91), ([['22423']], 91), ([['22424']], 90), ([['22427']], 91), ([['22429']], 91), ([['22431']], 91), ([['22432']], 91), ([['22456']], 91), ([['22457']], 91), ([['22464']], 91), ([['22467']], 91), ([['22469']], 91), ([['22470']], 91), ([['22487']], 91), ([['22488']], 91), ([['22489']], 91), ([['22492']], 91), ([['22499']], 91), ([['22501']], 91), ([['22502']], 91), ([['22505']], 91), ([['22507']], 91), ([['22549']], 91), ([['22551']], 91), ([['22553']], 91), ([['22554']], 91), ([['22555']], 91), ([['22556']], 91), ([['22557']], 91), ([['22558']], 91), ([['22560']], 91), ([['22561']], 91), ([['22568']], 91), ([['22569']], 91), ([['22570']], 91), ([['22577']], 91), ([['22578']], 91), ([['22579']], 91), ([['22585']], 91), ([['22595']], 91), ([['22607']], 91), ([['22616']], 91), ([['22617']], 91), ([['22619']], 91), ([['22620']], 91), ([['22621']], 91), ([['22624']], 91), ([['22625']], 91), ([['22627']], 91), ([['22629']], 91), ([['22630']], 91), ([['22631']], 91), ([['22632']], 91), ([['22633']], 91), ([['22637']], 91), ([['22652']], 91), ([['22659']], 91), ([['22661']], 91), ([['22662']], 91), ([['22663']], 91), ([['22666']], 91), ([['22667']], 91), ([['22668']], 91), ([['22690']], 91), ([['22692']], 91), ([['22694']], 91), ([['22697']], 91), ([['22698']], 91), ([['22699']], 91), ([['22720']], 91), ([['22721']], 91), ([['22722']], 91), ([['22723']], 91), ([['22725']], 91), ([['22726']], 91), ([['22727']], 91), ([['22728']], 91), ([['22729']], 91), ([['22730']], 91), ([['22734']], 91), ([['22745']], 91), ([['22748']], 91), ([['22749']], 91), ([['22750']], 91), ([['22752']], 91), ([['22759']], 91), ([['22766']], 91), ([['22771']], 91), ([['22776']], 91), ([['22784']], 91), ([['22795']], 91), ([['22804']], 91), ([['22834']], 91), ([['22835']], 91), ([['22844']], 91), ([['22846']], 91), ([['22847']], 91), ([['22851']], 91), ([['22865']], 91), ([['22866']], 91), ([['22867']], 91), ([['22896']], 90), ([['22898']], 91), ([['22899']], 91), ([['22900']], 91), ([['22907']], 91), ([['22908']], 91), ([['22909']], 91), ([['22910']], 91), ([['22940']], 91), ([['22941']], 91), ([['22951']], 91), ([['22952']], 91), ([['22957']], 91), ([['22960']], 91), ([['22961']], 91), ([['22962']], 91), ([['22963']], 91), ([['22964']], 91), ([['22966']], 91), ([['22969']], 91), ([['22970']], 91), ([['22971']], 91), ([['22979']], 91), ([['22980']], 91), ([['22983']], 91), ([['22988']], 91), ([['22989']], 91), ([['22991']], 91), ([['22992']], 91), ([['22993']], 91), ([['22996']], 91), ([['22998']], 91), ([['23076']], 91), ([['23077']], 91), ([['23082']], 91), ([['23084']], 91), ([['23100']], 91), ([['23101']], 91), ([['23103']], 91), ([['23108']], 91), ([['23109']], 91), ([['23110']], 91), ([['23126']], 91), ([['23144']], 91), ([['23146']], 90), ([['23147']], 91), ([['23165']], 90), ([['23169']], 91), ([['23170']], 91), ([['23171']], 91), ([['23172']], 91), ([['23173']], 91), ([['23174']], 91), ([['23175']], 91), ([['23188']], 91), ([['23191']], 91), ([['23192']], 91), ([['23196']], 91), ([['23198']], 91), ([['23199']], 91), ([['23200']], 91), ([['23201']], 91), ([['23202']], 91), ([['23203']], 91), ([['23204']], 91), ([['23205']], 91), ([['23206']], 91), ([['23207']], 91), ([['23208']], 91), ([['23209']], 91), ([['23230']], 91), ([['23231']], 91), ([['23232']], 91), ([['23234']], 91), ([['23236']], 91), ([['23238']], 91), ([['23240']], 91), ([['23241']], 91), ([['23243']], 91), ([['23245']], 91), ([['23247']], 91), ([['23254']], 91), ([['23256']], 91), ([['23263']], 91), ([['23264']], 91), ([['23265']], 91), ([['23266']], 91), ([['23275']], 91), ([['23284']], 91), ([['23293']], 91), ([['23294']], 91), ([['23295']], 91), ([['23296']], 91), ([['23298']], 91), ([['23300']], 91), ([['23301']], 91), ([['23306']], 91), ([['23307']], 91), ([['23308']], 91), ([['23309']], 91), ([['23311']], 91), ([['23312']], 91), ([['23313']], 91), ([['23318']], 91), ([['23319']], 91), ([['23320']], 91), ([['23321']], 91), ([['23322']], 91), ([['23323']], 91), ([['23328']], 91), ([['23329']], 91), ([['23332']], 91), ([['23333']], 91), ([['23343']], 91), ([['23344']], 91), ([['23349']], 91), ([['23350']], 91), ([['23351']], 91), ([['23353']], 91), ([['23354']], 91), ([['23355']], 91), ([['23356']], 91), ([['23367']], 91), ([['23439']], 91), ([['23493']], 91), ([['23497']], 90), ([['23503']], 91), ([['23534']], 91), ([['23570']], 91), ([['23571']], 91), ([['23581']], 91), ([['23582']], 91), ([['23583']], 91), ([['35970']], 91), ([['47566']], 91), ([['47566B']], 91), ([['47590A']], 91), ([['47590B']], 91), ([['47591D']], 91), ([['48138']], 91), ([['48184']], 91), ([['48187']], 91), ([['48188']], 91), ([['48194']], 91), ([['71459']], 91), ([['79321']], 91), ([['82482']], 91), ([['82483']], 91), ([['82484']], 91), ([['82486']], 91), ([['82494L']], 91), ([['82551']], 91), ([['82552']], 91), ([['82580']], 91), ([['82581']], 91), ([['82582']], 91), ([['82583']], 91), ([['82600']], 91), ([['84029E']], 91), ([['84029G']], 91), ([['84030E']], 91), ([['84077']], 91), ([['84347']], 91), ([['84375']], 91), ([['84378']], 91), ([['84380']], 91), ([['84596B']], 91), ([['84692']], 91), ([['84755']], 91), ([['84836']], 91), ([['84879']], 91), ([['84945']], 91), ([['84946']], 91), ([['84970L']], 91), ([['84970S']], 91), ([['84978']], 91), ([['84987']], 91), ([['84988']], 91), ([['84991']], 91), ([['84992']], 91), ([['84997B']], 91), ([['84997C']], 91), ([['84997D']], 91), ([['85014A']], 91), ([['85014B']], 91), ([['85049A']], 91), ([['85049E']], 91), ([['85066']], 91), ([['85099B']], 91), ([['85099C']], 91), ([['85099F']], 91), ([['85123A']], 91), ([['85150']], 91), ([['85152']], 91), ([['20712', '85099B']], 91), ([['20723', '20724']], 91), ([['20724', '20725']], 91), ([['20724', '22356']], 91), ([['20724', '85099B']], 91), ([['20725', '20726']], 91), ([['20725', '20727']], 91), ([['20725', '20728']], 91), ([['20725', '21212']], 91), ([['20725', '22382']], 91), ([['20725', '22383']], 91), ([['20725', '22384']], 91), ([['20725', '22386']], 91), ([['20725', '22662']], 91), ([['20725', '23203']], 91), ([['20725', '23206']], 91), ([['20725', '23207']], 91), ([['20725', '23208']], 91), ([['20725', '23209']], 91), ([['20725', '85099B']], 91), ([['20726', '20727']], 91), ([['20726', '20728']], 91), ([['20726', '22382']], 91), ([['20726', '22383']], 91), ([['20726', '22384']], 91), ([['20726', '23206']], 91), ([['20726', '23209']], 91), ([['20727', '20728']], 91), ([['20727', '22382']], 91), ([['20727', '22383']], 91), ([['20727', '22384']], 91), ([['20727', '22662']], 91), ([['20727', '23203']], 91), ([['20727', '23206']], 91), ([['20727', '23207']], 91), ([['20727', '23208']], 91), ([['20727', '23209']], 91), ([['20727', '85099B']], 91), ([['20728', '22382']], 91), ([['20728', '22383']], 91), ([['20728', '22384']], 91), ([['20728', '22662']], 91), ([['20728', '23206']], 91), ([['20728', '23207']], 91), ([['20728', '23209']], 91), ([['20728', '85099B']], 91), ([['21166', '21175']], 91), ([['21212', '21213']], 90), ([['21212', '21975']], 91), ([['21212', '21977']], 91), ([['21212', '84991']], 91), ([['21212', '84992']], 91), ([['21485', '22111']], 91), ([['21733', '85123A']], 91), ([['21754', '21755']], 91), ([['21914', '21915']], 91), ([['21928', '21929']], 91), ([['21928', '85099B']], 91), ([['21929', '22386']], 91), ([['21929', '22411']], 91), ([['21929', '85099B']], 91), ([['21930', '85099B']], 91), ([['21931', '22383']], 91), ([['21931', '85099B']], 91), ([['21977', '84991']], 91), ([['22086', '22910']], 91), ([['22112', '22114']], 91), ([['22112', '22835']], 91), ([['22112', '23355']], 90), ([['22138', '22617']], 91), ([['22141', '22144']], 91), ([['22382', '22383']], 91), ([['22382', '22384']], 91), ([['22382', '22662']], 91), ([['22382', '23206']], 91), ([['22382', '23207']], 91), ([['22382', '23209']], 91), ([['22382', '85099B']], 91), ([['22383', '22384']], 91), ([['22383', '22662']], 91), ([['22383', '23206']], 91), ([['22383', '23207']], 91), ([['22383', '23208']], 91), ([['22383', '23209']], 91), ([['22383', '85099B']], 91), ([['22384', '22386']], 91), ([['22384', '23203']], 91), ([['22384', '23206']], 91), ([['22384', '23207']], 91), ([['22384', '23209']], 91), ([['22384', '85099B']], 91), ([['22385', '85099B']], 91), ([['22386', '23203']], 91), ([['22386', '85099B']], 91), ([['22386', '85099F']], 91), ([['22411', '23203']], 91), ([['22411', '85099B']], 91), ([['22423', '22697']], 91), ([['22423', '22698']], 91), ([['22423', '22699']], 91), ([['22423', '23245']], 91), ([['22423', '85123A']], 90), ([['22457', '85123A']], 91), ([['22469', '22470']], 91), ([['22469', '85123A']], 91), ([['22470', '85123A']], 91), ([['22551', '22554']], 91), ([['22577', '22578']], 91), ([['22578', '22579']], 91), ([['22624', '22625']], 91), ([['22629', '22630']], 91), ([['22666', '22720']], 91), ([['22697', '22698']], 91), ([['22697', '22699']], 91), ([['22698', '22699']], 91), ([['22720', '22722']], 91), ([['22720', '23243']], 91), ([['22725', '22727']], 91), ([['22726', '22727']], 91), ([['22726', '22728']], 91), ([['22726', '22730']], 91), ([['22727', '22728']], 91), ([['22727', '22730']], 91), ([['22728', '22730']], 91), ([['22804', '85123A']], 91), ([['22865', '22866']], 91), ([['22865', '22867']], 91), ([['22865', '23439']], 91), ([['22960', '22961']], 91), ([['23170', '23171']], 91), ([['23170', '23172']], 90), ([['23171', '23172']], 91), ([['23173', '23175']], 91), ([['23174', '23175']], 91), ([['23199', '23200']], 91), ([['23199', '23201']], 91), ([['23199', '23202']], 91), ([['23199', '23203']], 91), ([['23199', '23206']], 91), ([['23199', '23209']], 91), ([['23199', '85099B']], 91), ([['23200', '23202']], 91), ([['23200', '23203']], 91), ([['23201', '23202']], 91), ([['23201', '23203']], 91), ([['23201', '23207']], 91), ([['23201', '23209']], 90), ([['23201', '85099B']], 91), ([['23202', '23203']], 91), ([['23202', '23208']], 91), ([['23202', '23209']], 91), ([['23202', '85099B']], 91), ([['23203', '23206']], 91), ([['23203', '23207']], 91), ([['23203', '23209']], 91), ([['23203', '23344']], 91), ([['23203', '85099B']], 91), ([['23206', '23207']], 91), ([['23206', '23208']], 91), ([['23206', '23209']], 91), ([['23206', '85099B']], 91), ([['23207', '23208']], 91), ([['23207', '23209']], 91), ([['23208', '23209']], 91), ([['23209', '85099B']], 91), ([['23298', '47566']], 91), ([['23300', '23301']], 91), ([['23321', '23322']], 91), ([['23343', '23344']], 91), ([['23355', '23356']], 91), ([['23581', '23582']], 91), ([['47566', '85123A']], 91), ([['47590A', '47590B']], 90), ([['82482', '82494L']], 91), ([['82482', '85123A']], 91), ([['82483', '82486']], 91), ([['82494L', '85123A']], 91), ([['84378', '84380']], 91), ([['84879', '85123A']], 91), ([['84991', '84992']], 91), ([['85099B', '85099C']], 91), ([['85099B', '85099F']], 91), ([['20685'], ['20685']], 91), ([['20712'], ['85099B']], 91), ([['20724'], ['20724']], 91), ([['20724'], ['20725']], 91), ([['20724'], ['23203']], 91), ([['20724'], ['23209']], 91), ([['20724'], ['85099B']], 91), ([['20725'], ['20724']], 91), ([['20725'], ['20725']], 91), ([['20725'], ['20726']], 91), ([['20725'], ['20727']], 91), ([['20725'], ['20728']], 91), ([['20725'], ['21212']], 91), ([['20725'], ['21931']], 91), ([['20725'], ['22086']], 91), ([['20725'], ['22382']], 91), ([['20725'], ['22383']], 91), ([['20725'], ['22384']], 91), ([['20725'], ['22385']], 90), ([['20725'], ['22386']], 91), ([['20725'], ['22411']], 91), ([['20725'], ['22423']], 91), ([['20725'], ['22662']], 91), ([['20725'], ['23084']], 91), ([['20725'], ['23199']], 91), ([['20725'], ['23200']], 90), ([['20725'], ['23201']], 91), ([['20725'], ['23202']], 91), ([['20725'], ['23203']], 91), ([['20725'], ['23206']], 91), ([['20725'], ['23207']], 91), ([['20725'], ['23208']], 91), ([['20725'], ['23209']], 91), ([['20725'], ['23298']], 91), ([['20725'], ['23344']], 91), ([['20725'], ['23355']], 91), ([['20725'], ['23581']], 91), ([['20725'], ['23582']], 91), ([['20725'], ['23583']], 91), ([['20725'], ['47566']], 91), ([['20725'], ['85099B']], 91), ([['20725'], ['85123A']], 91), ([['20726'], ['20725']], 91), ([['20726'], ['20726']], 91), ([['20726'], ['20727']], 91), ([['20726'], ['20728']], 91), ([['20726'], ['22382']], 91), ([['20726'], ['22383']], 91), ([['20726'], ['22384']], 91), ([['20726'], ['23203']], 91), ([['20726'], ['23206']], 91), ([['20726'], ['23207']], 91), ([['20726'], ['23208']], 91), ([['20726'], ['23209']], 91), ([['20726'], ['85099B']], 91), ([['20727'], ['20725']], 91), ([['20727'], ['20726']], 91), ([['20727'], ['20727']], 91), ([['20727'], ['20728']], 91), ([['20727'], ['21931']], 91), ([['20727'], ['22382']], 91), ([['20727'], ['22383']], 91), ([['20727'], ['22384']], 91), ([['20727'], ['22386']], 91), ([['20727'], ['23199']], 91), ([['20727'], ['23201']], 91), ([['20727'], ['23202']], 91), ([['20727'], ['23203']], 91), ([['20727'], ['23206']], 91), ([['20727'], ['23207']], 91), ([['20727'], ['23208']], 91), ([['20727'], ['23209']], 91), ([['20727'], ['23344']], 91), ([['20727'], ['23581']], 91), ([['20727'], ['23583']], 91), ([['20727'], ['85099B']], 91), ([['20727'], ['85123A']], 91), ([['20728'], ['20725']], 91), ([['20728'], ['20726']], 91), ([['20728'], ['20727']], 91), ([['20728'], ['20728']], 91), ([['20728'], ['21931']], 91), ([['20728'], ['22382']], 91), ([['20728'], ['22383']], 91), ([['20728'], ['22384']], 91), ([['20728'], ['22386']], 91), ([['20728'], ['22662']], 91), ([['20728'], ['23199']], 91), ([['20728'], ['23201']], 91), ([['20728'], ['23202']], 91), ([['20728'], ['23203']], 91), ([['20728'], ['23206']], 91), ([['20728'], ['23207']], 91), ([['20728'], ['23208']], 91), ([['20728'], ['23209']], 91), ([['20728'], ['23344']], 91), ([['20728'], ['23355']], 91), ([['20728'], ['23581']], 91), ([['20728'], ['23583']], 91), ([['20728'], ['85099B']], 91), ([['20914'], ['20914']], 91), ([['21034'], ['21034']], 91), ([['21080'], ['20725']], 91), ([['21080'], ['21080']], 91), ([['21080'], ['21212']], 91), ([['21080'], ['85099B']], 90), ([['21166'], ['21166']], 91), ([['21166'], ['21175']], 91), ([['21175'], ['21175']], 91), ([['21175'], ['21181']], 91), ([['21181'], ['21175']], 91), ([['21181'], ['21181']], 91), ([['21212'], ['20725']], 91), ([['21212'], ['20727']], 91), ([['21212'], ['20728']], 90), ([['21212'], ['21080']], 91), ([['21212'], ['21212']], 91), ([['21212'], ['21977']], 91), ([['21212'], ['22086']], 91), ([['21212'], ['22382']], 91), ([['21212'], ['22423']], 91), ([['21212'], ['22720']], 91), ([['21212'], ['22952']], 91), ([['21212'], ['22993']], 91), ([['21212'], ['23203']], 91), ([['21212'], ['23207']], 91), ([['21212'], ['23209']], 91), ([['21212'], ['23245']], 91), ([['21212'], ['23298']], 91), ([['21212'], ['23307']], 91), ([['21212'], ['23355']], 91), ([['21212'], ['47566']], 91), ([['21212'], ['84991']], 91), ([['21212'], ['85099B']], 91), ([['21212'], ['85123A']], 91), ([['21232'], ['21232']], 91), ([['21733'], ['21733']], 91), ([['21733'], ['85123A']], 91), ([['21754'], ['21754']], 91), ([['21754'], ['85123A']], 91), ([['21755'], ['21754']], 91), ([['21755'], ['85123A']], 91), ([['21790'], ['21790']], 91), ([['21928'], ['23203']], 91), ([['21928'], ['85099B']], 91), ([['21929'], ['20725']], 91), ([['21929'], ['21929']], 91), ([['21929'], ['23203']], 91), ([['21929'], ['23209']], 90), ([['21929'], ['85099B']], 91), ([['21930'], ['85099B']], 91), ([['21931'], ['20725']], 91), ([['21931'], ['20727']], 91), ([['21931'], ['20728']], 91), ([['21931'], ['21931']], 91), ([['21931'], ['22383']], 91), ([['21931'], ['22386']], 91), ([['21931'], ['23199']], 91), ([['21931'], ['23201']], 91), ([['21931'], ['23202']], 91), ([['21931'], ['23203']], 91), ([['21931'], ['23206']], 91), ([['21931'], ['23209']], 91), ([['21931'], ['85099B']], 91), ([['21936'], ['20725']], 91), ([['21936'], ['85099B']], 91), ([['21977'], ['21212']], 91), ([['21977'], ['21977']], 91), ([['21977'], ['23307']], 91), ([['22077'], ['22077']], 91), ([['22086'], ['22086']], 91), ([['22086'], ['22910']], 91), ([['22111'], ['22111']], 91), ([['22112'], ['22112']], 91), ([['22112'], ['23355']], 91), ([['22138'], ['22138']], 91), ([['22139'], ['22139']], 91), ([['22178'], ['22178']], 91), ([['22197'], ['22197']], 91), ([['22355'], ['22355']], 91), ([['22382'], ['20725']], 91), ([['22382'], ['20726']], 91), ([['22382'], ['20727']], 91), ([['22382'], ['20728']], 91), ([['22382'], ['22382']], 91), ([['22382'], ['22383']], 91), ([['22382'], ['22384']], 91), ([['22382'], ['22385']], 91), ([['22382'], ['22386']], 90), ([['22382'], ['22662']], 91), ([['22382'], ['23084']], 90), ([['22382'], ['23199']], 91), ([['22382'], ['23201']], 91), ([['22382'], ['23202']], 91), ([['22382'], ['23203']], 91), ([['22382'], ['23206']], 91), ([['22382'], ['23207']], 91), ([['22382'], ['23208']], 91), ([['22382'], ['23209']], 91), ([['22382'], ['23344']], 91), ([['22382'], ['23583']], 91), ([['22382'], ['85099B']], 91), ([['22383'], ['20725']], 91), ([['22383'], ['20726']], 91), ([['22383'], ['20727']], 91), ([['22383'], ['20728']], 91), ([['22383'], ['21931']], 91), ([['22383'], ['22382']], 91), ([['22383'], ['22383']], 91), ([['22383'], ['22384']], 91), ([['22383'], ['22662']], 91), ([['22383'], ['23199']], 91), ([['22383'], ['23201']], 91), ([['22383'], ['23203']], 91), ([['22383'], ['23206']], 91), ([['22383'], ['23207']], 91), ([['22383'], ['23208']], 91), ([['22383'], ['23209']], 91), ([['22383'], ['23583']], 91), ([['22383'], ['85099B']], 91), ([['22384'], ['20725']], 91), ([['22384'], ['20726']], 91), ([['22384'], ['20727']], 91), ([['22384'], ['20728']], 91), ([['22384'], ['22382']], 91), ([['22384'], ['22383']], 91), ([['22384'], ['22384']], 91), ([['22384'], ['22386']], 91), ([['22384'], ['23199']], 91), ([['22384'], ['23201']], 91), ([['22384'], ['23202']], 91), ([['22384'], ['23203']], 91), ([['22384'], ['23206']], 91), ([['22384'], ['23207']], 91), ([['22384'], ['23208']], 91), ([['22384'], ['23209']], 91), ([['22384'], ['23298']], 90), ([['22384'], ['23344']], 91), ([['22384'], ['23581']], 91), ([['22384'], ['23583']], 91), ([['22384'], ['85099B']], 91), ([['22385'], ['23203']], 91), ([['22385'], ['85099B']], 91), ([['22386'], ['20725']], 91), ([['22386'], ['20727']], 91), ([['22386'], ['22384']], 91), ([['22386'], ['22386']], 91), ([['22386'], ['22411']], 90), ([['22386'], ['23199']], 91), ([['22386'], ['23201']], 91), ([['22386'], ['23202']], 91), ([['22386'], ['23203']], 91), ([['22386'], ['23206']], 91), ([['22386'], ['23209']], 91), ([['22386'], ['23344']], 91), ([['22386'], ['85099B']], 91), ([['22411'], ['20725']], 91), ([['22411'], ['22411']], 91), ([['22411'], ['23201']], 91), ([['22411'], ['23202']], 91), ([['22411'], ['23203']], 91), ([['22411'], ['23209']], 91), ([['22411'], ['23581']], 91), ([['22411'], ['85099B']], 91), ([['22423'], ['20725']], 91), ([['22423'], ['20728']], 91), ([['22423'], ['21212']], 91), ([['22423'], ['22086']], 91), ([['22423'], ['22423']], 91), ([['22423'], ['22697']], 91), ([['22423'], ['22698']], 91), ([['22423'], ['22699']], 91), ([['22423'], ['22720']], 91), ([['22423'], ['23084']], 91), ([['22423'], ['23170']], 91), ([['22423'], ['23173']], 91), ([['22423'], ['23203']], 91), ([['22423'], ['23209']], 91), ([['22423'], ['23245']], 91), ([['22423'], ['23298']], 91), ([['22423'], ['23355']], 91), ([['22423'], ['47566']], 91), ([['22423'], ['84879']], 91), ([['22423'], ['85099B']], 91), ([['22423'], ['85123A']], 91), ([['22457'], ['22457']], 91), ([['22457'], ['85123A']], 91), ([['22469'], ['22469']], 91), ([['22469'], ['22470']], 91), ([['22469'], ['23298']], 91), ([['22469'], ['23321']], 91), ([['22469'], ['23322']], 91), ([['22469'], ['85099B']], 91), ([['22469'], ['85123A']], 91), ([['22470'], ['22469']], 91), ([['22470'], ['22470']], 91), ([['22470'], ['23321']], 91), ([['22470'], ['23322']], 91), ([['22470'], ['85123A']], 91), ([['22551'], ['22551']], 91), ([['22624'], ['22624']], 91), ([['22629'], ['22629']], 91), ([['22630'], ['22629']], 90), ([['22662'], ['20725']], 91), ([['22662'], ['20727']], 91), ([['22662'], ['20728']], 90), ([['22662'], ['22382']], 91), ([['22662'], ['22383']], 91), ([['22662'], ['22662']], 91), ([['22662'], ['23207']], 91), ([['22662'], ['23209']], 91), ([['22666'], ['22666']], 91), ([['22666'], ['22720']], 91), ([['22666'], ['22993']], 91), ([['22697'], ['22423']], 91), ([['22697'], ['22697']], 91), ([['22697'], ['22698']], 91), ([['22697'], ['22699']], 91), ([['22698'], ['22423']], 91), ([['22698'], ['22699']], 91), ([['22699'], ['22423']], 91), ([['22699'], ['22697']], 91), ([['22699'], ['22698']], 91), ([['22699'], ['22699']], 91), ([['22699'], ['23245']], 91), ([['22720'], ['20725']], 91), ([['22720'], ['21212']], 91), ([['22720'], ['22086']], 91), ([['22720'], ['22423']], 91), ([['22720'], ['22666']], 91), ([['22720'], ['22720']], 91), ([['22720'], ['22722']], 91), ([['22720'], ['22993']], 91), ([['22720'], ['23203']], 91), ([['22720'], ['23209']], 91), ([['22720'], ['23243']], 91), ([['22720'], ['23245']], 91), ([['22720'], ['23298']], 91), ([['22720'], ['23355']], 91), ([['22720'], ['47566']], 91), ([['22720'], ['85099B']], 91), ([['22720'], ['85123A']], 91), ([['22722'], ['22720']], 91), ([['22722'], ['22722']], 91), ([['22722'], ['22993']], 90), ([['22722'], ['23243']], 91), ([['22726'], ['22726']], 91), ([['22726'], ['22727']], 91), ([['22726'], ['22728']], 91), ([['22727'], ['22726']], 91), ([['22727'], ['22727']], 91), ([['22727'], ['22728']], 91), ([['22727'], ['22730']], 91), ([['22728'], ['22726']], 91), ([['22728'], ['22727']], 91), ([['22728'], ['22728']], 91), ([['22730'], ['22727']], 91), ([['22730'], ['22730']], 90), ([['22804'], ['85123A']], 91), ([['22865'], ['22865']], 91), ([['22910'], ['22086']], 91), ([['22910'], ['22910']], 91), ([['22960'], ['22086']], 90), ([['22960'], ['22720']], 91), ([['22960'], ['22960']], 91), ([['22960'], ['22993']], 91), ([['22960'], ['23209']], 91), ([['22960'], ['85099B']], 91), ([['22961'], ['20725']], 91), ([['22961'], ['21212']], 91), ([['22961'], ['22086']], 91), ([['22961'], ['22423']], 91), ([['22961'], ['22720']], 91), ([['22961'], ['22960']], 90), ([['22961'], ['22961']], 91), ([['22961'], ['22993']], 91), ([['22961'], ['23203']], 91), ([['22961'], ['23209']], 91), ([['22961'], ['23298']], 91), ([['22961'], ['47566']], 91), ([['22961'], ['85099B']], 91), ([['22993'], ['22993']], 91), ([['22993'], ['23209']], 91), ([['23084'], ['23084']], 91), ([['23199'], ['20725']], 91), ([['23199'], ['22382']], 91), ([['23199'], ['23199']], 91), ([['23199'], ['23200']], 91), ([['23199'], ['23201']], 91), ([['23199'], ['23202']], 91), ([['23199'], ['23203']], 91), ([['23199'], ['23206']], 91), ([['23199'], ['23209']], 91), ([['23199'], ['23344']], 91), ([['23199'], ['23581']], 91), ([['23199'], ['85099B']], 91), ([['23200'], ['23199']], 90), ([['23200'], ['23203']], 91), ([['23200'], ['85099B']], 91), ([['23201'], ['20725']], 91), ([['23201'], ['20727']], 91), ([['23201'], ['22382']], 91), ([['23201'], ['22383']], 90), ([['23201'], ['23199']], 91), ([['23201'], ['23200']], 91), ([['23201'], ['23201']], 91), ([['23201'], ['23202']], 91), ([['23201'], ['23203']], 91), ([['23201'], ['23206']], 91), ([['23201'], ['23207']], 91), ([['23201'], ['23209']], 91), ([['23201'], ['23344']], 91), ([['23201'], ['23581']], 91), ([['23201'], ['85099B']], 91), ([['23202'], ['20725']], 91), ([['23202'], ['23199']], 91), ([['23202'], ['23202']], 91), ([['23202'], ['23203']], 91), ([['23202'], ['23206']], 91), ([['23202'], ['23209']], 91), ([['23202'], ['23344']], 91), ([['23202'], ['23581']], 90), ([['23202'], ['85099B']], 91), ([['23203'], ['20725']], 91), ([['23203'], ['20727']], 91), ([['23203'], ['20728']], 91), ([['23203'], ['22086']], 91), ([['23203'], ['22382']], 91), ([['23203'], ['22383']], 91), ([['23203'], ['22384']], 91), ([['23203'], ['22386']], 91), ([['23203'], ['22411']], 91), ([['23203'], ['23199']], 91), ([['23203'], ['23201']], 91), ([['23203'], ['23202']], 91), ([['23203'], ['23203']], 91), ([['23203'], ['23206']], 91), ([['23203'], ['23208']], 91), ([['23203'], ['23209']], 91), ([['23203'], ['23343']], 91), ([['23203'], ['23344']], 91), ([['23203'], ['23581']], 91), ([['23203'], ['23582']], 91), ([['23203'], ['23583']], 91), ([['23203'], ['85099B']], 91), ([['23206'], ['20725']], 91), ([['23206'], ['20726']], 91), ([['23206'], ['20727']], 91), ([['23206'], ['20728']], 91), ([['23206'], ['22382']], 91), ([['23206'], ['22383']], 91), ([['23206'], ['22384']], 91), ([['23206'], ['23199']], 91), ([['23206'], ['23201']], 91), ([['23206'], ['23202']], 91), ([['23206'], ['23203']], 91), ([['23206'], ['23206']], 91), ([['23206'], ['23207']], 91), ([['23206'], ['23208']], 91), ([['23206'], ['23209']], 91), ([['23206'], ['23344']], 91), ([['23206'], ['23583']], 90), ([['23206'], ['85099B']], 91), ([['23207'], ['20725']], 91), ([['23207'], ['20726']], 90), ([['23207'], ['20727']], 91), ([['23207'], ['20728']], 91), ([['23207'], ['22382']], 91), ([['23207'], ['22383']], 91), ([['23207'], ['22384']], 91), ([['23207'], ['23199']], 90), ([['23207'], ['23201']], 91), ([['23207'], ['23202']], 91), ([['23207'], ['23203']], 91), ([['23207'], ['23206']], 91), ([['23207'], ['23207']], 91), ([['23207'], ['23208']], 91), ([['23207'], ['23209']], 91), ([['23207'], ['23344']], 91), ([['23207'], ['85099B']], 91), ([['23208'], ['20725']], 91), ([['23208'], ['20727']], 91), ([['23208'], ['22382']], 91), ([['23208'], ['22383']], 91), ([['23208'], ['22384']], 91), ([['23208'], ['23202']], 91), ([['23208'], ['23203']], 91), ([['23208'], ['23206']], 91), ([['23208'], ['23207']], 91), ([['23208'], ['23208']], 91), ([['23208'], ['23209']], 91), ([['23208'], ['85099B']], 91), ([['23209'], ['20725']], 91), ([['23209'], ['20726']], 91), ([['23209'], ['20727']], 91), ([['23209'], ['20728']], 91), ([['23209'], ['22382']], 91), ([['23209'], ['22383']], 91), ([['23209'], ['22384']], 91), ([['23209'], ['22386']], 91), ([['23209'], ['23199']], 91), ([['23209'], ['23201']], 91), ([['23209'], ['23202']], 91), ([['23209'], ['23203']], 91), ([['23209'], ['23206']], 91), ([['23209'], ['23207']], 91), ([['23209'], ['23208']], 91), ([['23209'], ['23209']], 91), ([['23209'], ['23344']], 91), ([['23209'], ['23581']], 91), ([['23209'], ['23582']], 91), ([['23209'], ['23583']], 91), ([['23209'], ['85099B']], 91), ([['23245'], ['22423']], 91), ([['23245'], ['23245']], 91), ([['23284'], ['23284']], 91), ([['23284'], ['23355']], 91), ([['23298'], ['20725']], 91), ([['23298'], ['22086']], 91), ([['23298'], ['23203']], 91), ([['23298'], ['23209']], 91), ([['23298'], ['23298']], 91), ([['23298'], ['23313']], 91), ([['23298'], ['47566']], 91), ([['23298'], ['85099B']], 91), ([['23300'], ['23300']], 91), ([['23300'], ['23301']], 91), ([['23301'], ['23300']], 91), ([['23301'], ['23301']], 91), ([['23321'], ['23321']], 91), ([['23344'], ['23344']], 91), ([['23355'], ['23355']], 91), ([['47566'], ['20725']], 91), ([['47566'], ['20727']], 91), ([['47566'], ['20728']], 91), ([['47566'], ['21212']], 90), ([['47566'], ['22086']], 91), ([['47566'], ['22382']], 91), ([['47566'], ['22383']], 91), ([['47566'], ['22423']], 91), ([['47566'], ['22720']], 90), ([['47566'], ['22910']], 91), ([['47566'], ['23203']], 91), ([['47566'], ['23209']], 91), ([['47566'], ['23245']], 91), ([['47566'], ['23298']], 91), ([['47566'], ['23355']], 91), ([['47566'], ['47566']], 91), ([['47566'], ['85099B']], 91), ([['47566'], ['85123A']], 91), ([['48138'], ['48138']], 91), ([['82482'], ['82482']], 91), ([['82482'], ['82494L']], 91), ([['82482'], ['85123A']], 91), ([['82483'], ['82483']], 91), ([['82484'], ['82484']], 91), ([['82494L'], ['82482']], 91), ([['82494L'], ['82494L']], 91), ([['82494L'], ['85123A']], 91), ([['84755'], ['84755']], 91), ([['84879'], ['22086']], 91), ([['84879'], ['22423']], 91), ([['84879'], ['84879']], 91), ([['84879'], ['85123A']], 91), ([['84946'], ['84946']], 91), ([['84978'], ['84978']], 91), ([['84991'], ['21212']], 91), ([['84991'], ['84991']], 91), ([['84992'], ['21212']], 91), ([['85099B'], ['20712']], 91), ([['85099B'], ['20724']], 91), ([['85099B'], ['20725']], 91), ([['85099B'], ['20726']], 91), ([['85099B'], ['20727']], 91), ([['85099B'], ['20728']], 91), ([['85099B'], ['21212']], 91), ([['85099B'], ['21928']], 91), ([['85099B'], ['21929']], 91), ([['85099B'], ['21930']], 91), ([['85099B'], ['21931']], 91), ([['85099B'], ['22086']], 91), ([['85099B'], ['22382']], 91), ([['85099B'], ['22383']], 91), ([['85099B'], ['22384']], 91), ([['85099B'], ['22385']], 91), ([['85099B'], ['22386']], 91), ([['85099B'], ['22411']], 91), ([['85099B'], ['22423']], 91), ([['85099B'], ['23199']], 91), ([['85099B'], ['23200']], 91), ([['85099B'], ['23201']], 91), ([['85099B'], ['23202']], 91), ([['85099B'], ['23203']], 91), ([['85099B'], ['23206']], 91), ([['85099B'], ['23207']], 91), ([['85099B'], ['23208']], 91), ([['85099B'], ['23209']], 91), ([['85099B'], ['23298']], 91), ([['85099B'], ['23343']], 91), ([['85099B'], ['23344']], 91), ([['85099B'], ['23355']], 91), ([['85099B'], ['23581']], 91), ([['85099B'], ['23582']], 91), ([['85099B'], ['23583']], 91), ([['85099B'], ['47566']], 91), ([['85099B'], ['85099B']], 91), ([['85099B'], ['85099C']], 91), ([['85099B'], ['85099F']], 91), ([['85099B'], ['85123A']], 91), ([['85099C'], ['23203']], 91), ([['85099C'], ['85099B']], 91), ([['85099C'], ['85099C']], 91), ([['85099F'], ['20725']], 91), ([['85099F'], ['22386']], 90), ([['85099F'], ['23199']], 90), ([['85099F'], ['23201']], 91), ([['85099F'], ['23203']], 91), ([['85099F'], ['23209']], 91), ([['85099F'], ['85099B']], 91), ([['85099F'], ['85099F']], 91), ([['85123A'], ['20725']], 91), ([['85123A'], ['20727']], 91), ([['85123A'], ['20728']], 91), ([['85123A'], ['21212']], 91), ([['85123A'], ['21733']], 91), ([['85123A'], ['21754']], 91), ([['85123A'], ['22086']], 91), ([['85123A'], ['22382']], 91), ([['85123A'], ['22383']], 91), ([['85123A'], ['22384']], 91), ([['85123A'], ['22423']], 91), ([['85123A'], ['22457']], 91), ([['85123A'], ['22469']], 91), ([['85123A'], ['22470']], 91), ([['85123A'], ['22720']], 91), ([['85123A'], ['22804']], 90), ([['85123A'], ['23203']], 91), ([['85123A'], ['23207']], 91), ([['85123A'], ['23209']], 91), ([['85123A'], ['23245']], 91), ([['85123A'], ['23298']], 91), ([['85123A'], ['23301']], 91), ([['85123A'], ['23321']], 91), ([['85123A'], ['23322']], 91), ([['85123A'], ['23344']], 90), ([['85123A'], ['23355']], 91), ([['85123A'], ['47566']], 91), ([['85123A'], ['82482']], 91), ([['85123A'], ['82494L']], 91), ([['85123A'], ['84879']], 91), ([['85123A'], ['85099B']], 91), ([['85123A'], ['85123A']], 91), ([['85152'], ['85152']], 91), ([['20725'], ['20725'], ['20725']], 91), ([['20725'], ['20725'], ['20726']], 91), ([['20725'], ['20725'], ['20727']], 91), ([['20725'], ['20725'], ['20728']], 91), ([['20725'], ['20725'], ['22382']], 91), ([['20725'], ['20725'], ['22383']], 91), ([['20725'], ['20725'], ['22384']], 91), ([['20725'], ['20725'], ['23203']], 91), ([['20725'], ['20725'], ['23206']], 91), ([['20725'], ['20725'], ['23207']], 91), ([['20725'], ['20725'], ['23209']], 91), ([['20725'], ['20725'], ['85099B']], 91), ([['20725'], ['20726'], ['20725']], 91), ([['20725'], ['20726'], ['20727']], 90), ([['20725'], ['20727'], ['20725']], 91), ([['20725'], ['20727'], ['20727']], 91), ([['20725'], ['20727'], ['22384']], 91), ([['20725'], ['20727'], ['23209']], 91), ([['20725'], ['20727'], ['85099B']], 91), ([['20725'], ['20728'], ['20725']], 91), ([['20725'], ['20728'], ['20727']], 91), ([['20725'], ['20728'], ['20728']], 91), ([['20725'], ['20728'], ['23209']], 91), ([['20725'], ['22382'], ['20725']], 91), ([['20725'], ['22382'], ['20727']], 91), ([['20725'], ['22382'], ['22382']], 91), ([['20725'], ['22382'], ['22383']], 91), ([['20725'], ['22382'], ['23209']], 91), ([['20725'], ['22383'], ['20725']], 91), ([['20725'], ['22383'], ['20727']], 91), ([['20725'], ['22383'], ['20728']], 90), ([['20725'], ['22383'], ['22382']], 91), ([['20725'], ['22383'], ['22383']], 91), ([['20725'], ['22383'], ['22384']], 91), ([['20725'], ['22383'], ['23206']], 91), ([['20725'], ['22383'], ['23209']], 91), ([['20725'], ['22384'], ['20725']], 91), ([['20725'], ['22384'], ['20727']], 91), ([['20725'], ['22384'], ['22383']], 91), ([['20725'], ['22384'], ['22384']], 91), ([['20725'], ['22384'], ['23209']], 91), ([['20725'], ['23203'], ['20725']], 91), ([['20725'], ['23203'], ['23203']], 91), ([['20725'], ['23203'], ['23209']], 91), ([['20725'], ['23203'], ['85099B']], 91), ([['20725'], ['23206'], ['20725']], 91), ([['20725'], ['23206'], ['20727']], 91), ([['20725'], ['23206'], ['22382']], 90), ([['20725'], ['23206'], ['23206']], 91), ([['20725'], ['23206'], ['23209']], 91), ([['20725'], ['23207'], ['20725']], 91), ([['20725'], ['23207'], ['23209']], 90), ([['20725'], ['23209'], ['20725']], 91), ([['20725'], ['23209'], ['20727']], 91), ([['20725'], ['23209'], ['22382']], 91), ([['20725'], ['23209'], ['23203']], 91), ([['20725'], ['23209'], ['23206']], 91), ([['20725'], ['23209'], ['23209']], 91), ([['20725'], ['23209'], ['85099B']], 91), ([['20725'], ['85099B'], ['20725']], 91), ([['20725'], ['85099B'], ['20727']], 91), ([['20725'], ['85099B'], ['23203']], 91), ([['20725'], ['85099B'], ['23209']], 90), ([['20725'], ['85099B'], ['85099B']], 91), ([['20725', '20726'], ['20725']], 91), ([['20725', '20726'], ['20726']], 91), ([['20725', '20726'], ['20727']], 91), ([['20725', '20726'], ['22382']], 91), ([['20725', '20726'], ['22383']], 91), ([['20725', '20726'], ['22384']], 91), ([['20725', '20726'], ['23206']], 91), ([['20725', '20726'], ['23209']], 91), ([['20725', '20727'], ['20725']], 91), ([['20725', '20727'], ['20727']], 91), ([['20725', '20727'], ['20728']], 91), ([['20725', '20727'], ['22382']], 91), ([['20725', '20727'], ['22383']], 91), ([['20725', '20727'], ['22384']], 91), ([['20725', '20727'], ['23203']], 90), ([['20725', '20727'], ['23206']], 91), ([['20725', '20727'], ['23209']], 91), ([['20725', '20727'], ['85099B']], 91), ([['20725', '20728'], ['20725']], 91), ([['20725', '20728'], ['20727']], 91), ([['20725', '20728'], ['20728']], 91), ([['20725', '20728'], ['22382']], 91), ([['20725', '20728'], ['22383']], 91), ([['20725', '20728'], ['22384']], 91), ([['20725', '20728'], ['23206']], 91), ([['20725', '20728'], ['23209']], 91), ([['20725', '20728'], ['85099B']], 90), ([['20725', '22382'], ['20725']], 91), ([['20725', '22382'], ['20727']], 91), ([['20725', '22382'], ['22382']], 91), ([['20725', '22382'], ['22383']], 91), ([['20725', '22382'], ['23206']], 90), ([['20725', '22382'], ['23209']], 91), ([['20725', '22383'], ['20725']], 91), ([['20725', '22383'], ['20727']], 91), ([['20725', '22383'], ['20728']], 91), ([['20725', '22383'], ['22382']], 91), ([['20725', '22383'], ['22383']], 91), ([['20725', '22383'], ['22384']], 91), ([['20725', '22383'], ['23206']], 91), ([['20725', '22383'], ['23209']], 91), ([['20725', '22383'], ['85099B']], 91), ([['20725', '22384'], ['20725']], 91), ([['20725', '22384'], ['20727']], 91), ([['20725', '22384'], ['20728']], 91), ([['20725', '22384'], ['22382']], 91), ([['20725', '22384'], ['22383']], 91), ([['20725', '22384'], ['22384']], 91), ([['20725', '22384'], ['23203']], 91), ([['20725', '22384'], ['23206']], 91), ([['20725', '22384'], ['23209']], 91), ([['20725', '22384'], ['85099B']], 91), ([['20725', '23206'], ['23206']], 91), ([['20725', '23209'], ['20725']], 91), ([['20725', '23209'], ['23203']], 91), ([['20725', '23209'], ['23209']], 91), ([['20725', '85099B'], ['20725']], 91), ([['20725', '85099B'], ['20727']], 91), ([['20725', '85099B'], ['23203']], 91), ([['20725', '85099B'], ['23209']], 91), ([['20725', '85099B'], ['85099B']], 91), ([['20726'], ['20725'], ['20725']], 91), ([['20726'], ['20726'], ['20725']], 91), ([['20726', '22382'], ['20725']], 91), ([['20726', '22382'], ['20726']], 90), ([['20726', '22382'], ['22382']], 91), ([['20727'], ['20725'], ['20725']], 91), ([['20727'], ['20725'], ['20727']], 91), ([['20727'], ['20725'], ['22384']], 90), ([['20727'], ['20725'], ['23209']], 91), ([['20727'], ['20727'], ['20725']], 91), ([['20727'], ['20727'], ['20727']], 91), ([['20727'], ['20727'], ['20728']], 91), ([['20727'], ['20727'], ['22383']], 91), ([['20727'], ['20727'], ['22384']], 91), ([['20727'], ['20727'], ['23209']], 91), ([['20727'], ['20728'], ['20725']], 91), ([['20727'], ['20728'], ['20727']], 91), ([['20727'], ['22383'], ['20727']], 91), ([['20727'], ['22384'], ['20725']], 91), ([['20727'], ['22384'], ['20727']], 91), ([['20727'], ['22384'], ['22384']], 91), ([['20727'], ['23206'], ['20727']], 91), ([['20727'], ['23209'], ['20725']], 91), ([['20727'], ['23209'], ['20727']], 91), ([['20727'], ['23209'], ['23209']], 91), ([['20727', '20728'], ['20725']], 91), ([['20727', '20728'], ['20727']], 91), ([['20727', '20728'], ['20728']], 91), ([['20727', '20728'], ['22383']], 91), ([['20727', '22382'], ['20725']], 91), ([['20727', '22382'], ['20727']], 91), ([['20727', '22383'], ['20725']], 91), ([['20727', '22383'], ['20727']], 91), ([['20727', '22383'], ['22383']], 91), ([['20727', '22383'], ['23206']], 90), ([['20727', '22384'], ['20725']], 91), ([['20727', '22384'], ['20727']], 91), ([['20727', '22384'], ['20728']], 90), ([['20727', '22384'], ['22384']], 91), ([['20728'], ['20725'], ['20725']], 91), ([['20728'], ['20725'], ['20727']], 91), ([['20728'], ['20725'], ['20728']], 91), ([['20728'], ['20725'], ['22383']], 91), ([['20728'], ['20725'], ['22384']], 91), ([['20728'], ['20725'], ['23209']], 91), ([['20728'], ['20725'], ['85099B']], 91), ([['20728'], ['20727'], ['20725']], 91), ([['20728'], ['20727'], ['20727']], 91), ([['20728'], ['20728'], ['20725']], 91), ([['20728'], ['20728'], ['20727']], 91), ([['20728'], ['20728'], ['20728']], 91), ([['20728'], ['22383'], ['20725']], 91), ([['20728'], ['22383'], ['20727']], 91), ([['20728'], ['22383'], ['22383']], 91), ([['20728'], ['22383'], ['22384']], 91), ([['20728'], ['22384'], ['20725']], 91), ([['20728'], ['22384'], ['20727']], 91), ([['20728'], ['22384'], ['22384']], 91), ([['20728'], ['23209'], ['20725']], 91), ([['20728'], ['23209'], ['20727']], 90), ([['20728'], ['23209'], ['23209']], 91), ([['20728'], ['85099B'], ['20725']], 91), ([['20728', '22382'], ['20725']], 91), ([['20728', '22383'], ['20725']], 91), ([['20728', '22383'], ['20727']], 91), ([['20728', '22383'], ['20728']], 91), ([['20728', '22383'], ['22383']], 91), ([['20728', '22383'], ['23209']], 91), ([['20728', '22384'], ['20725']], 91), ([['20728', '22384'], ['20727']], 91), ([['20728', '22384'], ['20728']], 91), ([['20728', '22384'], ['22383']], 91), ([['20728', '22384'], ['22384']], 91), ([['21733'], ['85123A'], ['85123A']], 91), ([['21733', '85123A'], ['85123A']], 91), ([['21929', '85099B'], ['85099B']], 90), ([['21931'], ['85099B'], ['85099B']], 91), ([['21931', '85099B'], ['23203']], 90), ([['21931', '85099B'], ['85099B']], 91), ([['22086', '22910'], ['22086']], 91), ([['22197'], ['22197'], ['22197']], 91), ([['22382'], ['20725'], ['20725']], 91), ([['22382'], ['20725'], ['20727']], 91), ([['22382'], ['20725'], ['22382']], 91), ([['22382'], ['20725'], ['23209']], 91), ([['22382'], ['22382'], ['22382']], 90), ([['22382'], ['22383'], ['22382']], 90), ([['22382'], ['22383'], ['22383']], 91), ([['22382'], ['23209'], ['20725']], 91), ([['22382'], ['23209'], ['22382']], 91), ([['22382'], ['23209'], ['23209']], 91), ([['22382', '22383'], ['20725']], 91), ([['22382', '22383'], ['20727']], 91), ([['22382', '22383'], ['22382']], 91), ([['22382', '22383'], ['22383']], 91), ([['22383'], ['20725'], ['20725']], 91), ([['22383'], ['20725'], ['20727']], 91), ([['22383'], ['20725'], ['22383']], 91), ([['22383'], ['20725'], ['23209']], 90), ([['22383'], ['20727'], ['20725']], 91), ([['22383'], ['20727'], ['20727']], 91), ([['22383'], ['20727'], ['22383']], 91), ([['22383'], ['20728'], ['20725']], 91), ([['22383'], ['20728'], ['22383']], 91), ([['22383'], ['22383'], ['20725']], 91), ([['22383'], ['22383'], ['20727']], 91), ([['22383'], ['22383'], ['20728']], 90), ([['22383'], ['22383'], ['22382']], 91), ([['22383'], ['22383'], ['22383']], 91), ([['22383'], ['22383'], ['22384']], 91), ([['22383'], ['22383'], ['23206']], 90), ([['22383'], ['22383'], ['23209']], 90), ([['22383'], ['22384'], ['20725']], 91), ([['22383'], ['22384'], ['20727']], 91), ([['22383'], ['22384'], ['22383']], 91), ([['22383'], ['23206'], ['22383']], 90), ([['22383'], ['23209'], ['20725']], 91), ([['22383'], ['23209'], ['20727']], 90), ([['22383'], ['23209'], ['22383']], 91), ([['22383'], ['23209'], ['23209']], 91), ([['22383', '22384'], ['20725']], 91), ([['22383', '22384'], ['22383']], 91), ([['22383', '23206'], ['22383']], 91), ([['22383', '23206'], ['23206']], 91), ([['22384'], ['20725'], ['20725']], 91), ([['22384'], ['20725'], ['20727']], 91), ([['22384'], ['20725'], ['22384']], 91), ([['22384'], ['20727'], ['20725']], 91), ([['22384'], ['20727'], ['20727']], 91), ([['22384'], ['22384'], ['20725']], 91), ([['22384'], ['22384'], ['20727']], 91), ([['22384'], ['22384'], ['22384']], 91), ([['22384'], ['22384'], ['85099B']], 90), ([['22384'], ['23209'], ['20725']], 91), ([['22384'], ['23209'], ['23209']], 91), ([['22384'], ['85099B'], ['23203']], 91), ([['22384'], ['85099B'], ['85099B']], 91), ([['22386'], ['22386'], ['85099B']], 91), ([['22386'], ['85099B'], ['22386']], 91), ([['22386'], ['85099B'], ['85099B']], 91), ([['22386', '85099B'], ['20725']], 91), ([['22386', '85099B'], ['22386']], 91), ([['22386', '85099B'], ['23203']], 91), ([['22386', '85099B'], ['85099B']], 91), ([['22411', '85099B'], ['85099B']], 91), ([['22423'], ['22423'], ['22423']], 91), ([['22423'], ['22423'], ['23245']], 91), ([['22469'], ['85123A'], ['85123A']], 91), ([['22469', '22470'], ['22469']], 91), ([['22697', '22699'], ['22423']], 91), ([['22697', '22699'], ['22697']], 91), ([['22697', '22699'], ['22698']], 91), ([['22697', '22699'], ['22699']], 91), ([['22720'], ['22720'], ['22720']], 91), ([['22720', '22722'], ['22720']], 90), ([['22726'], ['22726'], ['22726']], 90), ([['22726', '22727'], ['22726']], 91), ([['22726', '22727'], ['22727']], 91), ([['22727'], ['22726'], ['22726']], 90), ([['22727'], ['22727'], ['22726']], 91), ([['22727'], ['22727'], ['22727']], 91), ([['22727', '22728'], ['22727']], 91), ([['23199'], ['23203'], ['23203']], 91), ([['23199', '85099B'], ['85099B']], 90), ([['23202', '23203'], ['23202']], 91), ([['23202', '23203'], ['23203']], 91), ([['23202', '23203'], ['85099B']], 91), ([['23203'], ['23203'], ['23203']], 91), ([['23203'], ['23203'], ['23209']], 91), ([['23203'], ['23203'], ['23344']], 90), ([['23203'], ['23203'], ['23581']], 91), ([['23203'], ['23203'], ['23582']], 91), ([['23203'], ['23203'], ['85099B']], 91), ([['23203'], ['23209'], ['23203']], 91), ([['23203'], ['23209'], ['23209']], 91), ([['23203'], ['85099B'], ['23203']], 91), ([['23203'], ['85099B'], ['85099B']], 91), ([['23203', '23209'], ['20725']], 91), ([['23203', '23209'], ['23203']], 91), ([['23203', '23209'], ['23209']], 91), ([['23203', '23209'], ['85099B']], 91), ([['23203', '85099B'], ['20725']], 91), ([['23203', '85099B'], ['23203']], 91), ([['23203', '85099B'], ['85099B']], 91), ([['23206'], ['23203'], ['23203']], 91), ([['23206'], ['23206'], ['20725']], 91), ([['23206'], ['23206'], ['23206']], 91), ([['23206'], ['23206'], ['23209']], 90), ([['23206'], ['23209'], ['20725']], 90), ([['23206'], ['23209'], ['23206']], 90), ([['23206'], ['23209'], ['23209']], 91), ([['23206', '23207'], ['23206']], 91), ([['23206', '23207'], ['23209']], 91), ([['23206', '23209'], ['23206']], 91), ([['23206', '23209'], ['23209']], 91), ([['23207'], ['23209'], ['23209']], 91), ([['23207', '23209'], ['23209']], 91), ([['23208', '23209'], ['23209']], 90), ([['23209'], ['20725'], ['23209']], 91), ([['23209'], ['23203'], ['23203']], 91), ([['23209'], ['23203'], ['23209']], 91), ([['23209'], ['23209'], ['20725']], 91), ([['23209'], ['23209'], ['23203']], 91), ([['23209'], ['23209'], ['23209']], 91), ([['23298', '47566'], ['23298']], 91), ([['23298', '47566'], ['47566']], 91), ([['47566'], ['47566'], ['23298']], 90), ([['47566'], ['47566'], ['47566']], 91), ([['82482'], ['82482'], ['82494L']], 91), ([['82482'], ['82494L'], ['82494L']], 91), ([['82482', '82494L'], ['82482']], 91), ([['82482', '82494L'], ['82494L']], 91), ([['82494L'], ['82482'], ['82494L']], 91), ([['84879'], ['84879'], ['84879']], 91), ([['85099B'], ['20725'], ['20725']], 91), ([['85099B'], ['20725'], ['20727']], 91), ([['85099B'], ['20725'], ['23203']], 91), ([['85099B'], ['20725'], ['23209']], 91), ([['85099B'], ['20725'], ['85099B']], 91), ([['85099B'], ['20727'], ['20725']], 91), ([['85099B'], ['20727'], ['20727']], 91), ([['85099B'], ['20727'], ['23203']], 91), ([['85099B'], ['20727'], ['85099B']], 91), ([['85099B'], ['20728'], ['85099B']], 91), ([['85099B'], ['21931'], ['85099B']], 90), ([['85099B'], ['22384'], ['20725']], 90), ([['85099B'], ['22384'], ['23203']], 91), ([['85099B'], ['22384'], ['85099B']], 91), ([['85099B'], ['22386'], ['22386']], 91), ([['85099B'], ['22386'], ['23203']], 91), ([['85099B'], ['22386'], ['85099B']], 91), ([['85099B'], ['22411'], ['85099B']], 91), ([['85099B'], ['23199'], ['23199']], 90), ([['85099B'], ['23199'], ['23203']], 91), ([['85099B'], ['23199'], ['85099B']], 91), ([['85099B'], ['23201'], ['85099B']], 91), ([['85099B'], ['23202'], ['23202']], 90), ([['85099B'], ['23202'], ['23203']], 91), ([['85099B'], ['23202'], ['85099B']], 91), ([['85099B'], ['23203'], ['20725']], 91), ([['85099B'], ['23203'], ['23203']], 91), ([['85099B'], ['23203'], ['23209']], 91), ([['85099B'], ['23203'], ['23344']], 91), ([['85099B'], ['23203'], ['23581']], 91), ([['85099B'], ['23203'], ['23582']], 91), ([['85099B'], ['23203'], ['85099B']], 91), ([['85099B'], ['23206'], ['23203']], 91), ([['85099B'], ['23209'], ['23203']], 91), ([['85099B'], ['23209'], ['23209']], 91), ([['85099B'], ['23209'], ['85099B']], 91), ([['85099B'], ['85099B'], ['20725']], 91), ([['85099B'], ['85099B'], ['20727']], 91), ([['85099B'], ['85099B'], ['22386']], 91), ([['85099B'], ['85099B'], ['23199']], 91), ([['85099B'], ['85099B'], ['23201']], 91), ([['85099B'], ['85099B'], ['23202']], 91), ([['85099B'], ['85099B'], ['23203']], 91), ([['85099B'], ['85099B'], ['23209']], 91), ([['85099B'], ['85099B'], ['23344']], 91), ([['85099B'], ['85099B'], ['23581']], 91), ([['85099B'], ['85099B'], ['85099B']], 91), ([['85099B', '85099C'], ['85099B']], 91), ([['85099B', '85099F'], ['85099B']], 91), ([['85123A'], ['47566'], ['85123A']], 91), ([['85123A'], ['85123A'], ['21733']], 91), ([['85123A'], ['85123A'], ['22423']], 90), ([['85123A'], ['85123A'], ['82482']], 91), ([['85123A'], ['85123A'], ['85123A']], 91), ([['20725'], ['20725'], ['20725'], ['20725']], 91), ([['20725', '20727'], ['20725'], ['20727']], 91), ([['20725', '20727'], ['20727'], ['20725']], 91), ([['20725', '20727'], ['20727'], ['20727']], 91), ([['22386', '85099B'], ['85099B'], ['85099B']], 91), ([['22423'], ['22423'], ['22423'], ['22423']], 91), ([['85099B'], ['23203'], ['23203'], ['23203']], 91), ([['85099B'], ['23203'], ['23203'], ['85099B']], 91), ([['85099B'], ['23203'], ['85099B'], ['23203']], 90), ([['85099B'], ['85099B'], ['23203'], ['85099B']], 91), ([['85099B'], ['85099B'], ['85099B'], ['23203']], 91), ([['85099B'], ['85099B'], ['85099B'], ['85099B']], 91), ([['85123A'], ['85123A'], ['85123A'], ['85123A']], 91), ([['85123A'], ['85123A'], ['85123A'], ['85123A'], ['85123A']], 90)]
seq_list_120 = [([['20685']], 121), ([['20712']], 121), ([['20713']], 121), ([['20718']], 121), ([['20719']], 121), ([['20723']], 121), ([['20724']], 121), ([['20725']], 121), ([['20726']], 121), ([['20727']], 121), ([['20728']], 121), ([['20914']], 121), ([['20971']], 121), ([['20972']], 121), ([['21034']], 121), ([['21080']], 121), ([['21121']], 121), ([['21155']], 121), ([['21166']], 121), ([['21172']], 121), ([['21175']], 121), ([['21181']], 121), ([['21210']], 121), ([['21212']], 121), ([['21213']], 121), ([['21232']], 121), ([['21479']], 121), ([['21481']], 121), ([['21485']], 121), ([['21524']], 121), ([['21559']], 121), ([['21621']], 121), ([['21731']], 121), ([['21733']], 121), ([['21754']], 121), ([['21755']], 121), ([['21770']], 121), ([['21790']], 121), ([['21791']], 121), ([['21843']], 121), ([['21889']], 121), ([['21914']], 121), ([['21915']], 121), ([['21928']], 121), ([['21929']], 121), ([['21930']], 121), ([['21931']], 121), ([['21932']], 121), ([['21933']], 120), ([['21936']], 121), ([['21975']], 121), ([['21977']], 121), ([['21985']], 121), ([['22029']], 121), ([['22077']], 121), ([['22083']], 121), ([['22084']], 121), ([['22086']], 121), ([['22090']], 121), ([['22111']], 121), ([['22112']], 121), ([['22113']], 121), ([['22114']], 121), ([['22138']], 121), ([['22139']], 121), ([['22141']], 121), ([['22142']], 121), ([['22144']], 121), ([['22147']], 121), ([['22149']], 121), ([['22150']], 121), ([['22173']], 121), ([['22178']], 121), ([['22189']], 121), ([['22191']], 121), ([['22193']], 121), ([['22197']], 121), ([['22326']], 121), ([['22355']], 121), ([['22356']], 121), ([['22367']], 121), ([['22382']], 121), ([['22383']], 121), ([['22384']], 121), ([['22385']], 121), ([['22386']], 121), ([['22411']], 121), ([['22417']], 121), ([['22423']], 121), ([['22427']], 121), ([['22457']], 121), ([['22467']], 121), ([['22469']], 121), ([['22470']], 121), ([['22487']], 121), ([['22551']], 121), ([['22553']], 121), ([['22554']], 121), ([['22555']], 121), ([['22556']], 121), ([['22557']], 121), ([['22558']], 121), ([['22561']], 121), ([['22568']], 121), ([['22577']], 121), ([['22578']], 121), ([['22595']], 121), ([['22617']], 121), ([['22621']], 121), ([['22624']], 121), ([['22625']], 121), ([['22629']], 121), ([['22630']], 121), ([['22632']], 121), ([['22633']], 121), ([['22652']], 121), ([['22659']], 121), ([['22662']], 121), ([['22666']], 121), ([['22667']], 121), ([['22694']], 120), ([['22697']], 121), ([['22698']], 121), ([['22699']], 121), ([['22720']], 121), ([['22722']], 121), ([['22726']], 121), ([['22727']], 121), ([['22728']], 121), ([['22730']], 121), ([['22734']], 121), ([['22749']], 121), ([['22750']], 121), ([['22766']], 121), ([['22776']], 121), ([['22804']], 121), ([['22834']], 121), ([['22835']], 121), ([['22865']], 121), ([['22866']], 121), ([['22867']], 121), ([['22907']], 121), ([['22909']], 121), ([['22910']], 121), ([['22952']], 121), ([['22960']], 121), ([['22961']], 121), ([['22966']], 121), ([['22969']], 121), ([['22993']], 121), ([['22998']], 121), ([['23084']], 121), ([['23108']], 121), ([['23170']], 121), ([['23173']], 121), ([['23198']], 121), ([['23199']], 121), ([['23200']], 121), ([['23201']], 121), ([['23202']], 121), ([['23203']], 121), ([['23204']], 121), ([['23205']], 121), ([['23206']], 121), ([['23207']], 121), ([['23208']], 121), ([['23209']], 121), ([['23240']], 121), ([['23243']], 121), ([['23245']], 121), ([['23247']], 121), ([['23263']], 121), ([['23284']], 121), ([['23293']], 121), ([['23298']], 121), ([['23300']], 121), ([['23301']], 121), ([['23307']], 121), ([['23308']], 121), ([['23311']], 121), ([['23313']], 121), ([['23318']], 121), ([['23319']], 121), ([['23320']], 121), ([['23321']], 121), ([['23322']], 121), ([['23343']], 121), ([['23344']], 121), ([['23351']], 121), ([['23355']], 121), ([['23356']], 121), ([['23439']], 121), ([['23493']], 121), ([['23503']], 121), ([['23581']], 121), ([['23582']], 121), ([['23583']], 121), ([['35970']], 121), ([['47566']], 121), ([['47566B']], 121), ([['47591D']], 121), ([['48138']], 121), ([['48184']], 121), ([['48187']], 121), ([['48194']], 121), ([['71459']], 121), ([['82482']], 121), ([['82483']], 121), ([['82484']], 121), ([['82486']], 121), ([['82494L']], 121), ([['82583']], 121), ([['84029E']], 121), ([['84029G']], 120), ([['84077']], 121), ([['84375']], 121), ([['84378']], 121), ([['84380']], 121), ([['84692']], 121), ([['84755']], 121), ([['84836']], 121), ([['84879']], 121), ([['84946']], 121), ([['84970L']], 121), ([['84970S']], 121), ([['84978']], 121), ([['84991']], 121), ([['84992']], 121), ([['84997D']], 121), ([['85014B']], 120), ([['85049E']], 121), ([['85066']], 121), ([['85099B']], 121), ([['85099C']], 121), ([['85099F']], 121), ([['85123A']], 121), ([['85152']], 121), ([['20725', '20726']], 121), ([['20725', '20727']], 121), ([['20725', '20728']], 121), ([['20725', '22382']], 121), ([['20725', '22383']], 121), ([['20725', '22384']], 121), ([['20725', '23206']], 121), ([['20725', '23209']], 121), ([['20725', '85099B']], 121), ([['20726', '20727']], 121), ([['20726', '20728']], 121), ([['20726', '22382']], 121), ([['20726', '22383']], 121), ([['20727', '20728']], 121), ([['20727', '22382']], 121), ([['20727', '22383']], 121), ([['20727', '22384']], 121), ([['20727', '23206']], 121), ([['20727', '23209']], 121), ([['20728', '22382']], 121), ([['20728', '22383']], 121), ([['20728', '22384']], 121), ([['21212', '21977']], 121), ([['21212', '84991']], 121), ([['21733', '85123A']], 121), ([['21929', '85099B']], 121), ([['21931', '85099B']], 121), ([['22086', '22910']], 121), ([['22382', '22383']], 121), ([['22382', '22384']], 121), ([['22382', '23209']], 121), ([['22383', '22384']], 121), ([['22383', '23206']], 121), ([['22383', '23209']], 121), ([['22384', '23209']], 121), ([['22386', '85099B']], 121), ([['22423', '22697']], 121), ([['22423', '22699']], 121), ([['22469', '22470']], 121), ([['22577', '22578']], 121), ([['22697', '22698']], 121), ([['22697', '22699']], 121), ([['22698', '22699']], 121), ([['22720', '22722']], 121), ([['22726', '22727']], 121), ([['22727', '22728']], 121), ([['22727', '22730']], 121), ([['22960', '22961']], 121), ([['23199', '23203']], 121), ([['23199', '23206']], 121), ([['23199', '85099B']], 121), ([['23201', '23203']], 121), ([['23202', '23203']], 121), ([['23202', '85099B']], 121), ([['23203', '23209']], 121), ([['23203', '85099B']], 121), ([['23206', '23207']], 121), ([['23206', '23208']], 120), ([['23206', '23209']], 121), ([['23207', '23209']], 121), ([['23208', '23209']], 121), ([['23298', '47566']], 121), ([['23300', '23301']], 121), ([['82482', '82494L']], 121), ([['85099B', '85099F']], 121), ([['20725'], ['20725']], 121), ([['20725'], ['20726']], 121), ([['20725'], ['20727']], 121), ([['20725'], ['20728']], 121), ([['20725'], ['22382']], 121), ([['20725'], ['22383']], 121), ([['20725'], ['22384']], 121), ([['20725'], ['23203']], 121), ([['20725'], ['23206']], 121), ([['20725'], ['23207']], 121), ([['20725'], ['23208']], 121), ([['20725'], ['23209']], 121), ([['20725'], ['85099B']], 121), ([['20726'], ['20725']], 121), ([['20726'], ['20726']], 121), ([['20726'], ['20727']], 121), ([['20726'], ['22382']], 121), ([['20727'], ['20725']], 121), ([['20727'], ['20727']], 121), ([['20727'], ['20728']], 121), ([['20727'], ['22382']], 121), ([['20727'], ['22383']], 121), ([['20727'], ['22384']], 121), ([['20727'], ['23203']], 121), ([['20727'], ['23206']], 121), ([['20727'], ['23207']], 121), ([['20727'], ['23209']], 121), ([['20727'], ['85099B']], 121), ([['20728'], ['20725']], 121), ([['20728'], ['20727']], 121), ([['20728'], ['20728']], 121), ([['20728'], ['22382']], 121), ([['20728'], ['22383']], 121), ([['20728'], ['22384']], 121), ([['20728'], ['23203']], 121), ([['20728'], ['23206']], 121), ([['20728'], ['23207']], 121), ([['20728'], ['23209']], 121), ([['20728'], ['85099B']], 121), ([['21212'], ['20725']], 121), ([['21212'], ['21212']], 121), ([['21212'], ['23307']], 120), ([['21733'], ['85123A']], 121), ([['21754'], ['21754']], 121), ([['22086'], ['22086']], 121), ([['22197'], ['22197']], 121), ([['22382'], ['20725']], 121), ([['22382'], ['20726']], 120), ([['22382'], ['20727']], 121), ([['22382'], ['20728']], 121), ([['22382'], ['22382']], 121), ([['22382'], ['22383']], 121), ([['22382'], ['22384']], 121), ([['22382'], ['23203']], 120), ([['22382'], ['23209']], 121), ([['22383'], ['20725']], 121), ([['22383'], ['20727']], 121), ([['22383'], ['20728']], 121), ([['22383'], ['22382']], 121), ([['22383'], ['22383']], 121), ([['22383'], ['22384']], 121), ([['22383'], ['23203']], 121), ([['22383'], ['23206']], 121), ([['22383'], ['23209']], 121), ([['22384'], ['20725']], 121), ([['22384'], ['20727']], 121), ([['22384'], ['20728']], 121), ([['22384'], ['22382']], 121), ([['22384'], ['22383']], 121), ([['22384'], ['22384']], 121), ([['22384'], ['23203']], 121), ([['22384'], ['23206']], 121), ([['22384'], ['23207']], 121), ([['22384'], ['23209']], 121), ([['22384'], ['85099B']], 121), ([['22386'], ['22386']], 121), ([['22386'], ['23203']], 121), ([['22386'], ['85099B']], 121), ([['22423'], ['22423']], 121), ([['22423'], ['22699']], 121), ([['22423'], ['23245']], 121), ([['22423'], ['47566']], 121), ([['22423'], ['85123A']], 121), ([['22457'], ['22457']], 121), ([['22469'], ['22469']], 121), ([['22469'], ['23321']], 121), ([['22469'], ['85123A']], 121), ([['22470'], ['22470']], 121), ([['22470'], ['85123A']], 121), ([['22666'], ['22666']], 121), ([['22699'], ['22423']], 121), ([['22720'], ['22720']], 121), ([['22720'], ['23245']], 121), ([['22722'], ['22722']], 121), ([['22726'], ['22726']], 121), ([['22726'], ['22727']], 121), ([['22727'], ['22726']], 121), ([['22727'], ['22727']], 121), ([['22960'], ['22960']], 121), ([['22993'], ['22993']], 121), ([['23084'], ['23084']], 121), ([['23199'], ['23199']], 121), ([['23199'], ['23203']], 121), ([['23199'], ['85099B']], 121), ([['23201'], ['23201']], 121), ([['23201'], ['23203']], 121), ([['23201'], ['85099B']], 121), ([['23202'], ['23202']], 121), ([['23202'], ['23203']], 121), ([['23203'], ['20725']], 121), ([['23203'], ['23202']], 121), ([['23203'], ['23203']], 121), ([['23203'], ['23209']], 121), ([['23203'], ['23344']], 121), ([['23203'], ['23581']], 121), ([['23203'], ['85099B']], 121), ([['23206'], ['20725']], 121), ([['23206'], ['20727']], 121), ([['23206'], ['23203']], 121), ([['23206'], ['23206']], 121), ([['23206'], ['23209']], 121), ([['23207'], ['20725']], 121), ([['23207'], ['22382']], 120), ([['23207'], ['23207']], 121), ([['23207'], ['23209']], 121), ([['23209'], ['20725']], 121), ([['23209'], ['20727']], 121), ([['23209'], ['22382']], 121), ([['23209'], ['23203']], 121), ([['23209'], ['23206']], 121), ([['23209'], ['23209']], 121), ([['23209'], ['85099B']], 121), ([['23298'], ['23298']], 121), ([['23298'], ['47566']], 121), ([['47566'], ['22423']], 121), ([['47566'], ['23298']], 121), ([['47566'], ['47566']], 121), ([['82482'], ['82482']], 121), ([['82482'], ['82494L']], 121), ([['82494L'], ['82482']], 121), ([['82494L'], ['82494L']], 121), ([['84879'], ['84879']], 121), ([['85099B'], ['20725']], 121), ([['85099B'], ['20727']], 121), ([['85099B'], ['20728']], 121), ([['85099B'], ['22382']], 121), ([['85099B'], ['22384']], 121), ([['85099B'], ['22386']], 121), ([['85099B'], ['23199']], 121), ([['85099B'], ['23201']], 121), ([['85099B'], ['23202']], 121), ([['85099B'], ['23203']], 121), ([['85099B'], ['23206']], 121), ([['85099B'], ['23209']], 121), ([['85099B'], ['23344']], 121), ([['85099B'], ['23581']], 121), ([['85099B'], ['85099B']], 121), ([['85099F'], ['85099B']], 121), ([['85123A'], ['21733']], 121), ([['85123A'], ['22423']], 121), ([['85123A'], ['22469']], 121), ([['85123A'], ['22470']], 121), ([['85123A'], ['47566']], 121), ([['85123A'], ['82482']], 121), ([['85123A'], ['82494L']], 121), ([['85123A'], ['84879']], 121), ([['85123A'], ['85099B']], 121), ([['85123A'], ['85123A']], 121), ([['20725'], ['20725'], ['20725']], 121), ([['20725'], ['20725'], ['20727']], 121), ([['20725'], ['20727'], ['20725']], 121), ([['20725', '20727'], ['20725']], 121), ([['20725', '20727'], ['20727']], 121), ([['20725', '22383'], ['22383']], 121), ([['20725', '22384'], ['20725']], 121), ([['22423'], ['22423'], ['22423']], 121), ([['23203'], ['23203'], ['23203']], 121), ([['23203', '23209'], ['23203']], 121), ([['47566'], ['47566'], ['47566']], 121), ([['84879'], ['84879'], ['84879']], 121), ([['85099B'], ['20725'], ['85099B']], 121), ([['85099B'], ['23203'], ['23203']], 121), ([['85099B'], ['23203'], ['85099B']], 121), ([['85099B'], ['85099B'], ['23203']], 121), ([['85099B'], ['85099B'], ['85099B']], 121), ([['85123A'], ['85123A'], ['85123A']], 121), ([['85123A'], ['85123A'], ['85123A'], ['85123A']], 121)]
seq_list_150 = [([['20685']], 151), ([['20724']], 151), ([['20725']], 151), ([['20726']], 151), ([['20727']], 151), ([['20728']], 151), ([['20914']], 151), ([['20972']], 151), ([['21034']], 151), ([['21080']], 151), ([['21166']], 151), ([['21175']], 151), ([['21181']], 151), ([['21212']], 151), ([['21213']], 150), ([['21232']], 151), ([['21485']], 151), ([['21524']], 151), ([['21733']], 151), ([['21754']], 151), ([['21755']], 151), ([['21790']], 151), ([['21791']], 151), ([['21889']], 151), ([['21914']], 151), ([['21915']], 151), ([['21929']], 151), ([['21931']], 151), ([['21977']], 151), ([['22077']], 151), ([['22083']], 151), ([['22086']], 151), ([['22090']], 151), ([['22111']], 151), ([['22112']], 151), ([['22113']], 151), ([['22114']], 151), ([['22138']], 151), ([['22139']], 151), ([['22144']], 151), ([['22147']], 151), ([['22149']], 151), ([['22178']], 151), ([['22197']], 151), ([['22382']], 151), ([['22383']], 151), ([['22384']], 151), ([['22385']], 151), ([['22386']], 151), ([['22411']], 151), ([['22423']], 151), ([['22457']], 151), ([['22469']], 151), ([['22470']], 151), ([['22551']], 151), ([['22554']], 151), ([['22558']], 151), ([['22577']], 151), ([['22578']], 151), ([['22621']], 151), ([['22624']], 151), ([['22625']], 151), ([['22629']], 151), ([['22633']], 151), ([['22662']], 151), ([['22666']], 151), ([['22697']], 151), ([['22698']], 151), ([['22699']], 151), ([['22720']], 151), ([['22722']], 151), ([['22726']], 151), ([['22727']], 151), ([['22728']], 151), ([['22730']], 151), ([['22865']], 151), ([['22866']], 151), ([['22867']], 151), ([['22907']], 151), ([['22909']], 151), ([['22910']], 151), ([['22952']], 151), ([['22960']], 151), ([['22961']], 151), ([['22966']], 151), ([['22969']], 151), ([['22993']], 151), ([['23084']], 151), ([['23199']], 151), ([['23200']], 151), ([['23201']], 151), ([['23202']], 151), ([['23203']], 151), ([['23206']], 151), ([['23207']], 151), ([['23208']], 151), ([['23209']], 151), ([['23240']], 151), ([['23243']], 151), ([['23245']], 151), ([['23284']], 151), ([['23293']], 151), ([['23298']], 151), ([['23300']], 151), ([['23301']], 151), ([['23307']], 151), ([['23321']], 151), ([['23322']], 151), ([['23343']], 151), ([['23344']], 151), ([['23355']], 151), ([['23356']], 151), ([['23581']], 151), ([['23583']], 151), ([['47566']], 151), ([['48138']], 151), ([['48187']], 151), ([['48194']], 151), ([['82482']], 151), ([['82484']], 150), ([['82494L']], 151), ([['84378']], 151), ([['84380']], 151), ([['84755']], 151), ([['84879']], 151), ([['84946']], 151), ([['84970S']], 151), ([['84978']], 151), ([['84991']], 151), ([['84992']], 151), ([['85099B']], 151), ([['85099C']], 151), ([['85099F']], 151), ([['85123A']], 151), ([['85152']], 151), ([['20725', '20727']], 151), ([['20725', '20728']], 151), ([['20725', '22382']], 150), ([['20725', '22383']], 151), ([['20725', '22384']], 151), ([['20725', '23209']], 151), ([['20725', '85099B']], 151), ([['20727', '22384']], 151), ([['20728', '22383']], 150), ([['22086', '22910']], 151), ([['22386', '85099B']], 151), ([['22697', '22699']], 151), ([['22726', '22727']], 151), ([['23202', '23203']], 151), ([['23203', '23209']], 151), ([['23203', '85099B']], 151), ([['82482', '82494L']], 151), ([['20725'], ['20725']], 151), ([['20725'], ['20727']], 151), ([['20725'], ['20728']], 150), ([['20725'], ['22382']], 151), ([['20725'], ['22383']], 151), ([['20725'], ['22384']], 151), ([['20725'], ['23203']], 151), ([['20725'], ['23206']], 151), ([['20725'], ['23209']], 151), ([['20725'], ['85099B']], 151), ([['20727'], ['20725']], 151), ([['20727'], ['20727']], 151), ([['20728'], ['20725']], 151), ([['20728'], ['20728']], 151), ([['20728'], ['23209']], 151), ([['21212'], ['21212']], 151), ([['22382'], ['20725']], 151), ([['22382'], ['22382']], 150), ([['22383'], ['20725']], 151), ([['22383'], ['22383']], 151), ([['22384'], ['20725']], 151), ([['22384'], ['22384']], 151), ([['22423'], ['22423']], 151), ([['22469'], ['22469']], 151), ([['22720'], ['22720']], 151), ([['23203'], ['23203']], 151), ([['23203'], ['85099B']], 151), ([['23206'], ['23206']], 151), ([['23209'], ['23203']], 151), ([['23209'], ['23209']], 151), ([['23298'], ['23298']], 151), ([['47566'], ['47566']], 151), ([['84879'], ['84879']], 151), ([['85099B'], ['20725']], 151), ([['85099B'], ['23203']], 151), ([['85099B'], ['85099B']], 151), ([['85123A'], ['85123A']], 151), ([['22423'], ['22423'], ['22423']], 151), ([['85099B'], ['85099B'], ['85099B']], 151), ([['85123A'], ['85123A'], ['85123A']], 151)]
seq_list_200 = [([['20725']], 201), ([['20726']], 201), ([['20727']], 201), ([['20728']], 201), ([['21080']], 201), ([['21212']], 201), ([['21733']], 201), ([['21754']], 201), ([['21977']], 201), ([['22086']], 201), ([['22111']], 201), ([['22138']], 201), ([['22139']], 201), ([['22382']], 201), ([['22383']], 201), ([['22384']], 201), ([['22386']], 201), ([['22423']], 201), ([['22457']], 201), ([['22469']], 201), ([['22470']], 201), ([['22666']], 201), ([['22699']], 201), ([['22720']], 201), ([['22722']], 201), ([['22727']], 201), ([['22865']], 201), ([['22910']], 201), ([['22960']], 201), ([['22961']], 201), ([['22993']], 201), ([['23084']], 201), ([['23199']], 201), ([['23201']], 201), ([['23202']], 201), ([['23203']], 201), ([['23206']], 201), ([['23207']], 201), ([['23209']], 201), ([['23245']], 201), ([['23284']], 201), ([['23298']], 201), ([['23307']], 200), ([['23321']], 201), ([['23344']], 201), ([['23355']], 201), ([['47566']], 201), ([['82482']], 201), ([['82494L']], 201), ([['84879']], 201), ([['84991']], 201), ([['85099B']], 201), ([['85123A']], 201), ([['20725'], ['20725']], 201), ([['22423'], ['22423']], 201), ([['47566'], ['47566']], 201), ([['85099B'], ['85099B']], 201), ([['85123A'], ['85123A']], 201)]

In [83]:
#to keep low the next calculations
df_prod = df.loc[df.index,['ProdID','ProdDescr']]

In [77]:
#couple of functions to transform the above sequences in the same but more human readable (instead of ProdID have ProdDescr)
def get_prod_desc(seq_trans):
    prod_desc = []
    for trans in seq_trans:
        prod_desc_trans = []
        for item in trans:
            prod_desc_trans.append(df_prod[df['ProdID'] == item]['ProdDescr'].unique()[0])
        prod_desc.append(prod_desc_trans)
    return prod_desc

def get_prod_desc_seq(seq_trans_list):
    prod_desc_seq = []
    for seq_trans in tqdm(seq_trans_list, total=len(seq_trans_list)):
        seq_trans_prod_desc = get_prod_desc(seq_trans[0])
        prod_desc_seq.append((seq_trans_prod_desc,seq_trans[1]))
    return prod_desc_seq

In [78]:
seq_list_prod_90 = get_prod_desc_seq(seq_list_90)
seq_list_prod_120 = get_prod_desc_seq(seq_list_120)
seq_list_prod_150 = get_prod_desc_seq(seq_list_150)
seq_list_prod_200 = get_prod_desc_seq(seq_list_200)

### RESULTS

#### HAVING SUPPORT > 90 and printing only from above LEVEL 3

In [99]:
for trans in seq_list_prod_90:
    if len(trans[0]) > 3:
        print(trans[0])

[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT']]
[['REGENCY CAKESTAND 3 TIER'], ['REGENCY CAKESTAND 3 TIER'], ['REGENCY CAKESTAND 3 TIER'], ['REGENCY CAKESTAND 3 TIER']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG DOILEY PATTERNS']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG RED RETROSPOT']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG RED RETROSPOT']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT']]
[['WHITE HANGING HEART T-LIGHT HOLDER'], [

#### HAVING SUPPORT > 120 and printing only from above LEVEL 2

In [89]:
for trans in seq_list_prod_120:
    if len(trans[0]) > 2:
        print(trans[0])

[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG  BLACK SKULL.']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG  BLACK SKULL.'], ['LUNCH BAG RED RETROSPOT']]
[['REGENCY CAKESTAND 3 TIER'], ['REGENCY CAKESTAND 3 TIER'], ['REGENCY CAKESTAND 3 TIER']]
[['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG DOILEY PATTERNS']]
[['PARTY BUNTING'], ['PARTY BUNTING'], ['PARTY BUNTING']]
[['ASSORTED COLOUR BIRD ORNAMENT'], ['ASSORTED COLOUR BIRD ORNAMENT'], ['ASSORTED COLOUR BIRD ORNAMENT']]
[['JUMBO BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG DOILEY PATTERNS']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS'], ['JUMBO BAG RED RETROSPOT']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS']]
[['JUMBO BAG RED RETROSPOT'

#### HAVING SUPPORT > 150 and printing only from above LEVEL 1

In [94]:
for trans in seq_list_prod_150:
    if len(trans[0]) > 1:
        print(trans[0])

[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG  BLACK SKULL.']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG CARS BLUE']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG SPACEBOY DESIGN ']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG SUKI  DESIGN ']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG PINK POLKADOT']]
[['LUNCH BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG APPLE DESIGN']]
[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG DOILEY PATTERN ']]
[['LUNCH BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT']]
[['LUNCH BAG  BLACK SKULL.'], ['LUNCH BAG RED RETROSPOT']]
[['LUNCH BAG  BLACK SKULL.'], ['LUNCH BAG  BLACK SKULL.']]
[['LUNCH BAG CARS BLUE'], ['LUNCH BAG RED RETROSPOT']]
[['LUNCH BAG CARS BLUE'], ['LUNCH BAG CARS BLUE']]
[['LUNCH BAG CARS BLUE'], ['LUNCH BAG DOILEY PATTERN ']]
[['PACK OF 72 RETROSPOT CAKE CASES'], ['PACK OF 72 RETROSPOT CAKE CASES']]
[['LUNCH BAG SPACEBOY DESIGN '], ['LUNCH BAG RED RET

#### HAVING SUPPORT > 200 and printing only from above LEVEL 1

In [97]:
for trans in seq_list_prod_200:
    if len(trans[0]) > 1:
        print(trans[0])

[['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT']]
[['REGENCY CAKESTAND 3 TIER'], ['REGENCY CAKESTAND 3 TIER']]
[['PARTY BUNTING'], ['PARTY BUNTING']]
[['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT']]
[['WHITE HANGING HEART T-LIGHT HOLDER'], ['WHITE HANGING HEART T-LIGHT HOLDER']]


In [ ]:
# numero 16 fa paura con sup 10
#considerazioni su tutto l'insieme:
# con sup in range 80-99 tutto ok
# sup 70 is DEATH